In [1]:
import warnings; warnings.simplefilter('ignore')

In [3]:
from growing_instability_lib import *
# Start mem 476 MB

Slow version of gensim.models.doc2vec is being used
Using TensorFlow backend.


In [5]:
import gensim
gensim.models.word2vec.FAST_VERSION

-1

In [3]:
wvmodel = Word2Vec.load('corpus/train_body_data-with_labels_False-retain_special_chars_False.with_test_data.csv.w2v.model')

In [4]:
fsmodel = fasttext.load_model('corpus/train_body_data-with_labels_False-retain_special_chars_False.with_test_data.csv.fasttext.model.bin')

In [3]:
from encoder import Model
usnmodel = Model()

In [7]:
usnmodel.transform(transform_text(df.head()))

AttributeError: 'module' object has no attribute 'unescape'

In [120]:
%%time
_word2idx = {}

def word2idx(word):
    ix = len(_word2idx)
    _word2idx[word] = _word2idx.get(word, ix)
    return _word2idx[word]

def generate_training_word_vectors(stopwords=set()):
    processed_tname = set()
    nan_rows = set()
    target_columns = sorted(topics)
    wvec_trainingX = pd.DataFrame()
    fvec_trainingX = pd.DataFrame()
    word2idx_trainingX = pd.DataFrame()
    trainingY = pd.DataFrame()
    num_proc = 8

    for ix, tname in enumerate(glob.iglob('../data/TrainingData/*_TrainingData.json')):
        print('Processing {}. {}...'.format(ix + 1, tname))

        if tname in processed_tname:
            continue

        ds = parse_training_data_with_valid_topics(tname, topics=topics)
        ds = pd.DataFrame(ds).T

        for t in set(topics).difference(ds.columns):
            ds[t] = np.nan

        print('Done parsing {} docs with valid topics...'.format(ds.shape[0]))

        df_tokens = transform_text(ds)
        batch = min(df_tokens.shape[0] / num_proc, 2000)

        print('Computing fs features...')
        fvec = parallel_generate_word_vectors(df_tokens, transform_fasttext, stopwords=stopwords, batch=batch, num_proc=num_proc)

        print('Computing wv features...')
        wvec = parallel_generate_word_vectors(df_tokens, transform_word2vec, stopwords=stopwords, batch=batch, num_proc=num_proc)

        nulls = set()

        nulls.update(fvec[fvec.isnull()].index.values)
        nulls.update(wvec[wvec.isnull()].index.values)

        print 'Droping {} NaN samples...'.format(len(nulls))

        index = wvec.index.difference(nulls)
        wvec = wvec.ix[index]
        fvec = fvec.ix[index]

        if wvec_trainingX.empty:
            wvec_trainingX = wvec
        else:
            wvec_trainingX = wvec_trainingX.append(wvec)

        if fvec_trainingX.empty:
            fvec_trainingX = fvec
        else:
            fvec_trainingX = fvec_trainingX.append(fvec)

        print('Mapping word indices...')
        word_indices = df_tokens.ix[index].map(lambda x: [word2idx(i) for i in x.split()])

        if word2idx_trainingX.empty:
            word2idx_trainingX = word_indices
        else:
            word2idx_trainingX = word2idx_trainingX.append(word_indices)

        print('Current training size is {}...'.format(word2idx_trainingX.shape[0]))

        targets = ds[target_columns].ix[index]  # .fillna(0).as_matrix()

        if trainingY.empty:
            trainingY = targets
        else:
            trainingY = trainingY.append(targets)

        wvec_trainingX.to_hdf('training_data_wv_fs_no_stopwords', 'wvec_trainingX')
        fvec_trainingX.to_hdf('training_data_wv_fs_no_stopwords', 'fvec_trainingX')
        word2idx_trainingX.to_hdf('training_data_wv_fs_no_stopwords', 'word2idx_trainingX')
        pd.Series(_word2idx).to_hdf('training_data_wv_fs_no_stopwords', '_word2idx')
        trainingY.to_hdf('training_data_wv_fs_no_stopwords', 'trainingY_fasttext')

        print('Completed extracting data from {}...'.format(tname))
        print
        processed_tname.add(tname)

# CPU times: user 3h 11min 14s, sys: 2min 32s, total: 3h 13min 47s
# Wall time: 3h 12min 7s

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.91 µs


In [121]:
%%time
generate_training_word_vectors()

Processing 1. ../data/TrainingData/2003b_TrainingData.json...
Done parsing 6786 docs with valid topics...
Computing fs features...
Current batch in main thread: 5814
Current batch in main thread: 12597
Computing wv features...
Current batch in main thread: 5814
Current batch in main thread: 12597
Droping 0 NaN samples...
Mapping word indices...
Current training size is 6786...
Completed extracting data from ../data/TrainingData/2003b_TrainingData.json...

Processing 2. ../data/TrainingData/2004b_TrainingData.json...
Done parsing 7863 docs with valid topics...
Computing fs features...
Current batch in main thread: 6738
Current batch in main thread: 14599
Computing wv features...
Current batch in main thread: 6738
Current batch in main thread: 14599
Droping 0 NaN samples...
Mapping word indices...
Current training size is 14649...
Completed extracting data from ../data/TrainingData/2004b_TrainingData.json...

Processing 3. ../data/TrainingData/2005a_TrainingData.json...
Done parsing 6756

Completed extracting data from ../data/TrainingData/2001b_TrainingData.json...

Processing 19. ../data/TrainingData/2010b_TrainingData.json...
Done parsing 9561 docs with valid topics...
Computing fs features...
Current batch in main thread: 8190
Current batch in main thread: 17745
Computing wv features...
Current batch in main thread: 8190
Current batch in main thread: 17745
Droping 4 NaN samples...
Mapping word indices...
Current training size is 132578...
Completed extracting data from ../data/TrainingData/2010b_TrainingData.json...

Processing 20. ../data/TrainingData/2002b_TrainingData.json...
Done parsing 6180 docs with valid topics...
Computing fs features...
Current batch in main thread: 5292
Current batch in main thread: 11466
Computing wv features...
Current batch in main thread: 5292
Current batch in main thread: 11466
Droping 0 NaN samples...
Mapping word indices...
Current training size is 138758...
Completed extracting data from ../data/TrainingData/2002b_TrainingData.jso

In [125]:
trainingY = pd.read_hdf('training_data_wv_fs_no_stopwords', 'trainingY_fasttext')

In [142]:
tty = trainingY.copy()
freqs = tty.sum().sort_values(ascending=False)

In [137]:
freqs.head()

ukcrime        20513.0
iraq           20344.0
religion       16951.0
immigration    15296.0
london         15129.0
dtype: float64

In [130]:
freqs.describe()

count      146.000000
mean      2254.082192
std       3955.853032
min          3.000000
25%        212.750000
50%        630.000000
75%       2052.750000
max      20513.000000
dtype: float64

In [131]:
indices = resample_data(tty, upsample=100, maxsample=400)

In [143]:
tty.ix[indices].sum().sort_values(ascending=False)

humanrights                    2203.0
military                       2060.0
iraq                           1986.0
ukcrime                        1962.0
uksecurity                     1682.0
afghanistan                    1663.0
syria                          1595.0
immigration                    1546.0
police                         1506.0
london                         1435.0
unitednations                  1331.0
israel                         1311.0
religion                       1219.0
protest                        1198.0
terrorism                      1176.0
france                         1133.0
alqaida                        1131.0
defence                        1056.0
arabandmiddleeastprotests      1056.0
criminaljustice                 980.0
russia                          902.0
refugees                        897.0
aid                             893.0
india                           892.0
hacking                         876.0
surveillance                    869.0
naturaldisas

In [147]:
tty['year'] = tty.index.map(lambda x: re.findall('^([0-9]{4}).*', x)[0])

In [170]:
time_distribution = tty[freqs.head(len(freqs) - 14).index.union(['year'])].groupby('year').sum()

In [183]:
%matplotlib inline

time_distribution #.plot()

,ukcrime,iraq,religion,immigration,london,economy,israel,transport,afghanistan,france,...,internallydisplacedpeople,sydneysiege,cobra,encryption,southchinasea,darknet,hashtags,values,debate,australianguncontrol
year,,,,,,,,,,,,,,,,,,,,,
1999,246,59,172,42,177,18,91,182,35,NaN,...,3.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN
2000,335,67,316,263,374,37,595,538,8,1.0,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2001,481,122,329,738,347,326,868,770,2620,NaN,...,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN
2002,1008,1168,594,1309,391,678,1243,914,687,408.0,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003,1346,6098,593,1677,441,902,797,716,167,592.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004,1650,3066,777,2472,275,1206,882,771,199,659.0,...,NaN,NaN,NaN,NaN,4.0,1.0,NaN,NaN,NaN,NaN
2005,1495,1769,1072,1815,252,960,497,499,156,626.0,...,1.0,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN
2006,1731,1576,1211,1709,345,837,1133,767,374,541.0,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2007,2024,2039,1175,2034,448,967,849,1049,452,805.0,...,NaN,NaN,6.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN


In [176]:
unique_topic_articles = tty[freqs.index].sum(axis=1)
unique_topic_articles = unique_topic_articles[unique_topic_articles == 1]

In [182]:
tty.ix[unique_topic_articles.index][freqs.head(len(freqs) - 14).index.union(['year'])].groupby('year').sum()

,ukcrime,iraq,religion,immigration,london,economy,israel,transport,afghanistan,france,...,internallydisplacedpeople,sydneysiege,cobra,encryption,southchinasea,darknet,hashtags,values,debate,australianguncontrol
year,,,,,,,,,,,,,,,,,,,,,
1999,214,51,169,42,175,18,88,178,28,NaN,...,2.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN
2000,310,57,293,60,347,37,509,511,4,1.0,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2001,383,91,228,338,197,301,728,660,1436,NaN,...,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN
2002,621,869,467,696,264,633,1093,784,391,282.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003,868,4897,469,674,279,827,699,573,109,452.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004,1034,2420,578,1534,171,1087,740,587,151,481.0,...,NaN,NaN,NaN,NaN,4.0,1.0,NaN,NaN,NaN,NaN
2005,1079,1342,840,1006,166,894,442,406,103,552.0,...,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN
2006,1111,1165,933,858,255,740,528,657,148,448.0,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2007,1259,1332,912,1079,285,865,652,859,185,672.0,...,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN


In [181]:
unique_topic_articles

2003b_TrainingData_00068    1.0
2003b_TrainingData_00093    1.0
2003b_TrainingData_00100    1.0
2003b_TrainingData_00127    1.0
2003b_TrainingData_00132    1.0
2003b_TrainingData_00147    1.0
2003b_TrainingData_00149    1.0
2003b_TrainingData_00164    1.0
2003b_TrainingData_00212    1.0
2003b_TrainingData_00397    1.0
2003b_TrainingData_00408    1.0
2003b_TrainingData_00409    1.0
2003b_TrainingData_00410    1.0
2003b_TrainingData_00411    1.0
2003b_TrainingData_00424    1.0
2003b_TrainingData_00433    1.0
2003b_TrainingData_00436    1.0
2003b_TrainingData_00440    1.0
2003b_TrainingData_00454    1.0
2003b_TrainingData_00462    1.0
2003b_TrainingData_00463    1.0
2003b_TrainingData_00464    1.0
2003b_TrainingData_00469    1.0
2003b_TrainingData_00470    1.0
2003b_TrainingData_00472    1.0
2003b_TrainingData_00473    1.0
2003b_TrainingData_00474    1.0
2003b_TrainingData_00475    1.0
2003b_TrainingData_00478    1.0
2003b_TrainingData_00489    1.0
                           ... 
2011b_Tr

In [4]:
df = parse_training_data_with_valid_topics('../data/TrainingData/2014b_TrainingData.json', topics=topics)

In [5]:
df = pd.DataFrame(df).T

In [16]:
x = df[df.columns.intersection(topics)].sum(axis=1)
x[x.isnull()]

2014b_TrainingData_52907   NaN
dtype: float64

2014b_TrainingData_00001    today wasila tasi u has her day in court thoug...
2014b_TrainingData_00024    the ghana midfielder michael essien and his cl...
2014b_TrainingData_00041    oscar pistorius is facing possibly his last we...
2014b_TrainingData_00054    an exiled russian oligarch who was once so clo...
2014b_TrainingData_00135    head high or stay at home and if you go defini...
Name: bodyText, dtype: object